In [ ]:
pip install turicreate

In [ ]:
import turicreate as tc
import numpy as np

In [ ]:
sales = tc.SFrame('/content/drive/My Drive/home_data.sframe')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float and then to int, before using it below
sales['floors'] = sales['floors'].astype(float).astype(int)

In [ ]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = tc.SFrame()
    for feature in features:
      features_sframe[feature] = data_sframe[feature]
    features_sframe
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

In [ ]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix,weights)
    return(predictions)

In [ ]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
X

array([[ 3.,  5.,  8.],
       [ 4., 12., 15.]])

In [ ]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
norms

array([ 5., 13., 17.])

In [ ]:
X/norms

array([[0.6       , 0.38461538, 0.47058824],
       [0.8       , 0.92307692, 0.88235294]])

In [ ]:
def normalize_features(feature_matrix):
  norms = np.linalg.norm(feature_matrix, axis=0)
  normalized_features = feature_matrix/norms
  return (normalized_features, norms)


To test the function, run the following:

In [ ]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print(features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print(norms)
# should print
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


Implementing Coordinate Descent with normalized features



Let us consider a simple model with 2 features:

In [ ]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

In [ ]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

In [ ]:
simple_feature_matrix

array([[0.00680209, 0.00353021, 0.00583571],
       [0.00680209, 0.00768869, 0.00583571],
       [0.00680209, 0.00230361, 0.00389048],
       ...,
       [0.00680209, 0.00305154, 0.00389048],
       [0.00680209, 0.00478673, 0.00583571],
       [0.00680209, 0.00305154, 0.00389048]])

In [ ]:
weights = np.array([1., 4., 1.])

In [ ]:
prediction = predict_output(simple_feature_matrix,weights)
prediction

array([0.02675867, 0.04339256, 0.01990703, ..., 0.02289873, 0.03178473,
       0.02289873])


Compute the values of ro[i] for each feature in this simple model, using the formula given above, using the formula:

ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
Hint: You can get a Numpy vector for feature_i using:

simple_feature_matrix[:,i]

In [ ]:
ro = []
for i in range(len(weights)):
  value = simple_feature_matrix[:,i]*(output - prediction + weights[i]*simple_feature_matrix[:,i])
  ro.append(value.sum())
print(ro)



  


[79400300.03492916, 87939470.77299108, 80966698.67596565]


we have ro[0], ro[1], ro[2]

For W1 to be zero, we need ro[1] in [-lambda/2, lambda/2]

We have -lambda/2 <= ro[1] <= lambda/2

This translates to lambda >= -2ro[1] and lambda >= 2ro[1]

For both conditions to be satisfied, lambda >= 2ro[1] = 1.75e8

Similarly for W2. lambda >= 2ro[2] = 1.62e8.

So, w[i] = 0 if lambda >= 2 * abs(ro[i])

For quiz 1:

What range of values of l1_penalty would not set w[1] zero, but would set w[2] to zero, if we were to take a step in that coordinate?

Simply calculate the value for lambda that will set w2 to zero

i.e. a lambda value that is greater than 2*ro[2]

but that is less than the value that will set w1 to zero

i.e. a lambda value that is less then 2*ro[1]

This will be a range of lambda values between 2ro[2] and 2ro[1]

In [ ]:
print(2* ro[1])
print(2* ro[2])



175878941.54598215
161933397.3519313


In [ ]:
# Return True if value is within the threshold ranges otherwise False
# Looking for range -l1_penalty/2 <= ro <= l1_penalty/2
def in_l1range(value, penalty):
    return ( (value >= -penalty/2.) and (value <= penalty/2.) )



In [ ]:
for l1_penalty in [1.4e8, 1.64e8, 1.73e8, 1.9e8, 2.3e8]:
    print(in_l1range(ro[1], l1_penalty), in_l1range(ro[2], l1_penalty))

False False
False True
False True
True True
True True


In [ ]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix,weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:,i] * (output - prediction + (weights[i] * feature_matrix[:,i]))).sum()

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = (ro_i + l1_penalty/2.)
    elif ro_i > l1_penalty/2.:
        new_weight_i = (ro_i - l1_penalty/2.)
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [ ]:
# should print 0.425558846691
import math
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


In [ ]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
  weights = np.array(initial_weights)
  change = np.array(initial_weights) * 0.0
  converged = False
  
  while not converged:
    for i in range(len(weights)):
      old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
      # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
      #     and old values for weight[i], ..., weight[d-1]
      weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
      # use old_weights_i to compute change in coordinate
      change[i] = np.abs(weights[i] - old_weights_i)
    
    max_change = max(change)
    if max_change < tolerance:
      converged = True
  return weights

In [ ]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

In [ ]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

In [ ]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
weights

array([21624998.36636292, 63157246.78545421,        0.        ])

In [ ]:
normalized_simple_feature_matrix

array([[0.00680209, 0.00353021, 0.00583571],
       [0.00680209, 0.00768869, 0.00583571],
       [0.00680209, 0.00230361, 0.00389048],
       ...,
       [0.00680209, 0.00305154, 0.00389048],
       [0.00680209, 0.00478673, 0.00583571],
       [0.00680209, 0.00305154, 0.00389048]])

In [ ]:
weights

array([21624998.36636292, 63157246.78545421,        0.        ])

In [ ]:
predictions = predict_output(normalized_simple_feature_matrix,weights)
output = np.array(sales['price'])
error = output- predictions
value = error*error
value_1 = value.sum()
value_1



1630492481484487.8

Evaluating LASSO fit with more features

In [ ]:
train_data,test_data = sales.random_split(.8,seed=0)

In [ ]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']
my_output= 'price'

In [ ]:
(simple_feature_matrix_train, output_train) = get_numpy_data(train_data, all_features, my_output)
(normalized_simple_feature_matrix_train, simple_norms_train) = normalize_features(simple_feature_matrix_train) # normalize features

In [ ]:
initial_weights_train = np.zeros(14)
l1_penalty_train = 1e7
tolerance_train = 1.0

In [ ]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix_train, output_train,
                                            initial_weights_train, l1_penalty_train, tolerance_train)
weights1e7

array([24429600.60933314,        0.        ,        0.        ,
       48389174.35227978,        0.        ,        0.        ,
        3317511.16271981,  7329961.9848964 ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [ ]:
weights1e8 = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix_train, output_train,
                                            initial_weights_train, 1e8, 1)
weights1e8

array([71114625.75280938,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [ ]:
weights1e4 = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix_train, output_train,
                                            initial_weights_train, 1e4,5e5)
weights1e4

array([ 77779073.91265225, -22884012.25023361,  15348487.08089994,
        92166869.69883077,  -2139328.08242779,  -8818455.54409491,
         6494209.73310655,   7065162.05053198,   4119079.21006762,
        18436483.52618778, -14566678.54514345,  -5528348.75179427,
       -83591746.20730534,   2784276.46012858])

In [ ]:
def normalizing(weights,norms):
  weights_normalized = weights / norms
  return weights_normalized

In [ ]:
normalized_weights1e7 = normalizing(weights1e7,simple_norms_train)
normalized_weights1e4 = normalizing(weights1e4,simple_norms_train)
normalized_weights1e8 = normalizing(weights1e8,simple_norms_train)


Evaluating each of the learned models on the test data

In [ ]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

In [ ]:
def RSS(feature_matrix,weights):
  predictions = predict_output(feature_matrix,weights)
  output = np.array(test_data['price'])
  error = output - predictions
  value = (error*error).sum()
  return value

In [ ]:
print(RSS(test_feature_matrix,normalized_weights1e7))
print(RSS(test_feature_matrix,normalized_weights1e4))
print(RSS(test_feature_matrix,normalized_weights1e8))

275962079909185.28
227781004760225.34
537166150034084.9
